<a href="https://colab.research.google.com/github/mushamvineeth/Vineeth_INFO5731_Spring2021/blob/main/In_class_exercise_09/In_class_exercise_09.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **The ninth in-class-exercise (20 points in total, 4/16/2021)**

The purpose of the exercise is to practice different machine learning algorithms for text classification as well as the performance evaluation. In addition, you are requried to conduct *10 fold cross validation (https://scikit-learn.org/stable/modules/cross_validation.html)* in the training. 

The dataset can be download from here: https://github.com/unt-iialab/info5731_spring2021/blob/main/class_exercises/exercise09_datacollection.zip. The dataset contains two files train data and test data for sentiment analysis in IMDB review, it has two categories: 1 represents positive and 0 represents negative. You need to split the training data into training and validate data (80% for training and 20% for validation, https://towardsdatascience.com/train-test-split-and-cross-validation-in-python-80b61beca4b6) and perform 10 fold cross validation while training the classifier. The final trained model was final evaluated on the test data. 

Algorithms:

(1) MultinominalNB

(2) SVM 

(3) KNN 

(4) Decision tree

(5) Random Forest

(6) XGBoost

Evaluation measurement:

(1) Accuracy

(2) Recall

(3) Precison 

(4) F-1 score

In [106]:
# Write your code here
import nltk
nltk.download()


NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> all
       | 
       | Downloading package abc to /root/nltk_data...
       |   Package abc is already up-to-date!
       | Downloading package alpino to /root/nltk_data...
       |   Package alpino is already up-to-date!
       | Downloading package biocreative_ppi to /root/nltk_data...
       |   Package biocreative_ppi is already up-to-date!
       | Downloading package brown to /root/nltk_data...
       |   Package brown is already up-to-date!
       | Downloading package brown_tei to /root/nltk_data...
       |   Package brown_tei is already up-to-date!
       | Downloading package cess_cat to /root/nltk_data...
       |   Package cess_cat is already up-to-date!
       | Down

True

In [108]:
#Reading and formating training data set
import pandas as pd
train_data = open('/content/stsa-train.txt').read()
train_label, train_text = [], []
for i, j in enumerate(train_data.split("\n")):
    c = j.split(' ')
    train_label.append(c[0])
    train_text.append(" ".join(c[1:]))

In [109]:
#Reading and formating test data set
test_data = open('/content/stsa-test.txt').read()
test_label, test_text = [], []
for i, j in enumerate(test_data.split("\n")):
    c = j.split(' ')
    test_label.append(c[0])
    test_text.append(" ".join(c[1:]))

In [110]:
# creating data frame
df = pd.DataFrame (list(zip(train_text, train_label)) , columns = ['Raw Text', 'Labels'])
test_df = pd.DataFrame (list(zip(test_text, test_label)) , columns = ['Raw Text', 'Labels'])
test_df = test_df.dropna()

In [111]:
#cleaning train data set
import re
import nltk
from nltk.corpus import stopwords
from textblob import TextBlob
from nltk.stem import PorterStemmer
from textblob import Word
nltk.download('wordnet')
st = PorterStemmer()
nltk.download('punkt')
stop = stopwords.words('english')
#Lower case
df['After Lowercasing'] = df['Raw Text'].apply(lambda x: " ".join(x.lower() for x in x.split()))
#Removing Punctuation
df['After Removing Punctuation'] = df['After Lowercasing'].str.replace('[^\w\s]','')
#Removing special charachters
df['After Removing special charachters'] = df['After Removing Punctuation'].apply(lambda x: ''.join(re.sub(r"[^a-zA-Z0-9]+", ' ', charctr) for charctr in x ))
#Removing stopwords
df['After Removing Stopwords'] = df['After Removing Punctuation'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
#Tokenization
df['Tokenization'] = df['After Removing Stopwords'].apply(lambda x: TextBlob(x).words)
#Lemmatization
df['After Lemmatization'] = df['Tokenization'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x]))


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [112]:
#cleaning test data set

#Lower case
test_df['After Lowercasing'] = test_df['Raw Text'].apply(lambda x: " ".join(x.lower() for x in x.split()))
#Removing punctuation
test_df['After Removing Punctuation'] = test_df['After Lowercasing'].str.replace('[^\w\s]','')
#Removing special characters
test_df['After Removing Special Characters'] = test_df['After Removing Punctuation'].apply(lambda x: ''.join(re.sub(r"[^a-zA-Z0-9]+", ' ', charctr) for charctr in x ))
#Removing stopwords
test_df['After Removing Stopwords'] = test_df['After Removing Punctuation'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
#Tokenization
test_df['Tokenization'] = test_df['After Removing Stopwords'].apply(lambda x: TextBlob(x).words)
#Lemmatization
test_df['After Lemmatization'] = test_df['Tokenization'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x]))

In [113]:
#Data Transformation of train data set
from sklearn import model_selection, preprocessing, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer
train_idf_vect = TfidfVectorizer(analyzer='word')
train_idf_vect.fit(df['After Lemmatization'])
x_tfidf =  train_idf_vect.transform(df['After Lemmatization'])

In [87]:
#Data Tranformation of test data set
vect_test = TfidfVectorizer(analyzer='word', vocabulary = tfidf_vect.vocabulary_)
vect_test.fit(test_df['After Lemmatization'])
xtest = vect_test.transform(test_df['After Lemmatization'])
test_y = test_df['Labels']

In [114]:
#splitting of data into training and testing

from sklearn import model_selection, preprocessing, naive_bayes, metrics, svm
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(x_tfidf, df['Labels'].values,test_size=0.2)

from sklearn import metrics
def get_metrics(predictions, test_data_y):
  acc = metrics.accuracy_score(predictions, test_data_y)
  prec = metrics.precision_score(predictions, test_data_y, pos_label='positive', average='micro')
  recall = metrics.recall_score(predictions, test_data_y, pos_label='positive', average='micro')
  f1 = metrics.f1_score(predictions, test_data_y, pos_label='positive', average='micro')
  return acc, prec, recall, f1

import numpy as np
from sklearn.model_selection import cross_val_score, KFold
def cross_validation_score(modelName, x, y):
  scoring = 'accuracy'
  kfold = KFold(10, random_state = 7,shuffle=True)
  cross_val = cross_val_score(modelName, x, y, cv=kfold).mean()
  return cross_val

In [116]:
#Navie Bayes - Validation Metrics

nbm = naive_bayes.MultinomialNB()
nbm.fit(train_x, train_y)
nbpvdata = nbm.predict(valid_x)
acc, prec, recall, f1 = get_metrics(nbpvdata, valid_y)
print("Accuracy is ", acc,"\nPrecision is",prec,"\nRecall is:",recall,"\nF1 is:",f1)

#cross validation scores of Navie Bayes
nb_scores_mean = cross_validation_score(nbm, valid_x, valid_y)
print('Navie Bayes Cross Validation Score is ',nb_scores_mean)

#Testing Metrics
nbpt_data = nbm.predict(xtest)
acc, prec, recall, f1 = get_metrics(nbpt_data, test_y)
print("Accuracy is ", acc,"\nPrecision is",prec,"\nRecall is:",recall,"\nF1 is:",f1)

#cross validation scores of MultinominalNB_Testing
testing_mean = cross_validation_score(nbm, xtest, test_y)
print('Navie Bayes Cross Validation Score is ', testing_mean)



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1321: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  % (pos_label, average), UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1321: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  % (pos_label, average), UserWarning)


Accuracy is  0.7725631768953068 
Precision is 0.7725631768953068 
Recall is: 0.7725631768953068 
F1 is: 0.7725631768953068
Navie Bayes Cross Validation Score is  0.669393181107288
Accuracy is  0.7985729967069155 
Precision is 0.7985729967069155 
Recall is: 0.7985729967069155 
F1 is: 0.7985729967069155
Navie Bayes Cross Validation Score is  0.7371044256290158


In [118]:
#SVM - Validation Metrics
svm_model = svm.SVC()
svm_model.fit(train_x, train_y)
svm_predictions_validation_data = svm_model.predict(valid_x)
accuracy, precision, recall, f1 = get_metrics(svm_predictions_validation_data, valid_y)
print("Accuracy is ", accuracy,"\nPrecision is",precision,"\nRecall is:",recall,"\nF1 is:",f1)

#cross validation scores- svm
svm_mean = cross_validation_score(svm_model, valid_x, valid_y)
print('SVM Cross Validation Score is', svm_mean)

#Testing Metrics
svm_predictions_test_data = svm_model.predict(xtest)
accuracy, precision, recall, f1 = get_metrics(svm_predictions_test_data, test_y)
print("Accuracy is ", accuracy,"\nPrecision is",precision,"\nRecall is:",recall,"\nF1 is:",f1)

#cross validation scores- svm-testing
svm_test_scores_mean = cross_validation_score(svm_model, xtest, test_y)
print('SVM Cross Validation Score is {0}'.format(svm_test_scores_mean))

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1321: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  % (pos_label, average), UserWarning)


Accuracy is  0.7653429602888087 
Precision is 0.7653429602888087 
Recall is: 0.7653429602888087 
F1 is: 0.7653429602888087
SVM Cross Validation Score is 0.6441351266812637


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1321: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  % (pos_label, average), UserWarning)


Accuracy is  0.7947310647639956 
Precision is 0.7947310647639956 
Recall is: 0.7947310647639956 
F1 is: 0.7947310647639956
SVM Cross Validation Score is 0.7239086050561461


In [119]:
#KNN
from sklearn.neighbors import KNeighborsClassifier
knn_model = KNeighborsClassifier(n_neighbors = 15)
knn_model.fit(train_x, train_y)
knn_predictions_valid_data = knn_model.predict(valid_x)
accuracy, precision, recall, f1 = get_metrics(knn_predictions_valid_data, valid_y)
print("Accuracy is ", accuracy,"\nPrecision is",precision,"\nRecall is:",recall,"\nF1 is:",f1)

#cross validation score
knn_scores_mean = cross_validation_score(knn_model, valid_x, valid_y)
print('KNN Cross Validation Score is', knn_scores_mean)

#Testing Metrics
knn_predictions_test_data = knn_model.predict(xtest)
accuracy, precision, recall, f1 = get_metrics(knn_predictions_test_data, test_y)
print("Accuracy is ", accuracy,"\nPrecision is",precision,"\nRecall is:",recall,"\nF1 is:",f1)

#cross validation scores-Testing 
knn_test_scores_mean = cross_validation_score(knn_model, xtest, test_y)
print('KNN Cross Validation Score is', knn_test_scores_mean)

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1321: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  % (pos_label, average), UserWarning)


Accuracy is  0.6425992779783394 
Precision is 0.6425992779783394 
Recall is: 0.6425992779783394 
F1 is: 0.6425992779783394
KNN Cross Validation Score is 0.6672296945052654


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1321: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  % (pos_label, average), UserWarning)


Accuracy is  0.6866081229418222 
Precision is 0.6866081229418222 
Recall is: 0.6866081229418222 
F1 is: 0.6866081229418222
KNN Cross Validation Score is 0.6361376328589443


In [124]:
#Decision Tree
from sklearn.tree import DecisionTreeClassifier
decision_tree_model = DecisionTreeClassifier()
decision_tree_model.fit(train_x, train_y)
dt_predictions_valid_data = decision_tree_model.predict(valid_x)
accuracy, precision, recall, f1 = get_metrics(dt_predictions_valid_data, valid_y)
print("Accuracy is ", accuracy,"\nPrecision is",precision,"\nRecall is:",recall,"\nF1 is:",f1)

#cross validation scores
decision_tree_scores_mean = cross_validation_score(decision_tree_model, valid_x, valid_y)
print('Decision Tree Cross Validation Score is',decision_tree_scores_mean)

#Testing Metrics
decision_tree_predictions_test_data = decision_tree_model.predict(xtest)
accuracy, precision, recall, f1 = get_metrics(decision_tree_predictions_test_data, test_y)
print("Accuracy is ", accuracy,"\nPrecision is",precision,"\nRecall is:",recall,"\nF1 is:",f1)

#cross validation score testing
decision_tree_test_scores_mean = cross_validation_score(decision_tree_model, xtest, test_y)
print('Decision Tree Cross Validation Score is', decision_tree_test_scores_mean)

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1321: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  % (pos_label, average), UserWarning)


Accuracy is  0.6368231046931407 
Precision is 0.6368231046931407 
Recall is: 0.6368231046931407 
F1 is: 0.6368231046931407
Decision Tree Cross Validation Score is 0.5942446043165468
Accuracy is  0.6783754116355654 
Precision is 0.6783754116355654 
Recall is: 0.6783754116355654 
F1 is: 0.6783754116355654


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1321: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  % (pos_label, average), UserWarning)


Decision Tree Cross Validation Score is 0.6185372005044135


In [126]:
#Random Forest
from sklearn.ensemble import RandomForestClassifier
random_forest_model = RandomForestClassifier()
random_forest_model.fit(train_x, train_y)
random_forest_predictions_valid_data = random_forest_model.predict(valid_x)
accuracy, precision, recall, f1 = get_metrics(random_forest_predictions_valid_data, valid_y)
print("Accuracy is ", accuracy,"\nPrecision is",precision,"\nRecall is:",recall,"\nF1 is:",f1)

#cross validation score
random_forest_scores_mean = cross_validation_score(random_forest_model, valid_x, valid_y)
print('Random Forest Cross Validation Score is {0}'.format(random_forest_scores_mean))

#Testing Metrics
random_forest_predictions_test_data = random_forest_model.predict(xtest)
accuracy, precision, recall, f1 = get_metrics(random_forest_predictions_test_data, test_y)
print("Accuracy is ", accuracy,"\nPrecision is",precision,"\nRecall is:",recall,"\nF1 is:",f1)

#cross validation score testing
random_forest_test_scores_mean = cross_validation_score(random_forest_model, xtest, test_y)
print('Random Forest Cross Validation Score is', random_forest_test_scores_mean)

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1321: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  % (pos_label, average), UserWarning)


Accuracy is  0.7061371841155235 
Precision is 0.7061371841155235 
Recall is: 0.7061371841155235 
F1 is: 0.7061371841155235
Random Forest Cross Validation Score is 0.6203159211761026


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1321: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  % (pos_label, average), UserWarning)


Accuracy is  0.7338090010976949 
Precision is 0.7338090010976949 
Recall is: 0.7338090010976949 
F1 is: 0.7338090010976949
Random Forest Cross Validation Score is 0.6772443403590945


In [130]:
#XGBoost
from xgboost import XGBClassifier
xg_boost_model = XGBClassifier()
xg_boost_model.fit(train_x, train_y)
xg_boost_predictions_valid_data = xg_boost_model.predict(valid_x)
accuracy, precision, recall, f1 = get_metrics(xg_boost_predictions_valid_data, valid_y)
print("Accuracy is ", accuracy,"\nPrecision is",precision,"\nRecall is:",recall,"\nF1 is:",f1)

#cross validation score
xg_boost_scores_mean = cross_validation_score(XGBClassifier(), valid_x, valid_y)
print('XG Boost Cross Validation Score is ', xg_boost_scores_mean)

#Testing
xg_boost_predictions_test_data = xg_boost_model.predict(xtest)
accuracy, precision, recall, f1 = get_metrics(xg_boost_predictions_test_data, test_y)
print("Accuracy is ", accuracy,"\nPrecision is",precision,"\nRecall is:",recall,"\nF1 is:",f1)

#cross validation score testing
xg_boost_test_scores_mean = cross_validation_score(XGBClassifier(), xtest, test_y)
print('XG Boost Cross Validation Score is',xg_boost_test_scores_mean)

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1321: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  % (pos_label, average), UserWarning)


Accuracy is  0.6584837545126354 
Precision is 0.6584837545126354 
Recall is: 0.6584837545126354 
F1 is: 0.6584837545126354
XG Boost Cross Validation Score is  0.6051245959753936
Accuracy is  0.6448957189901208 
Precision is 0.6448957189901208 
Recall is: 0.6448957189901208 
F1 is: 0.6448957189901208
XG Boost Cross Validation Score is 0.6251035849396506
